In [33]:
import cv2

for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print('Webcam found at index: ' + str(i))
        cap.release()

Webcam found at index: 0
Webcam found at index: 1
Webcam found at index: 2


In [ ]:
import cv2
 
video = cv2.VideoCapture("rtmp://localhost:1935/live/123")
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)
size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print(size)
while True:
    ret, frame = video.read()
    cv2.imshow("A video", frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
video.release()
cv2.destroyAllWindows()

In [31]:
import face_recognition

# 加载第一张照片并识别面部特征
image1 = face_recognition.load_image_file("C:/Users/wsh/Pictures/Camera Roll/WIN_20240402_12_01_27_Pro.jpg")
encoding1 = face_recognition.face_encodings(image1)

# 加载第二张照片并识别面部特征
image2 = face_recognition.load_image_file("D:/Smoke/PictureHome/9/2023/1-25/no_face_11_1711965481.jpg")
encoding2 = face_recognition.face_encodings(image2)
print(len(encoding1))
print(len(encoding2))
# # 比较两个面部特征
# results = face_recognition.compare_faces([encoding1], encoding2)
# 
# if results[0]:
#     print("这两张照片是同一个人。")
# else:
#     print("这两张照片不是同一个人。")

0
0


In [5]:
import cProfile
import io
import pstats
import time
from pyexpat import model
import cv2
import numpy as np
import torch
from configList import *
from utils.augmentations import letterbox
from utils.general import check_img_size, non_max_suppression, scale_coords
from utils.plots import Annotator
WEIGHTS = 'weights/yolov5n.pt'
IMGSZ = [640, 640]  # 图像尺寸
CONF_THRES = 0.5  # 置信度阈值
IOU_THRES = 0.5  # IOU阈值
MAX_DET = 1000  # 最大检测数量
LINE_THICKNESS = 1  # 线条厚度
HIDE_CONF = False  # 是否隐藏置信度
HIDE_LABELS = None  # 是否隐藏标签
# model, device, half, stride, names = get_model()
# imgsz = check_img_size([640, 640], s=stride)
def colors(index, bright=True):
    color_list = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]
    return color_list[index % len(color_list)]
from models.experimental import attempt_load
from utils.torch_utils import select_device

WEIGHTS = "D:/Smoke/SmokeDetectionMasterPython/quantized_model.pt"
# WEIGHTS = "D:/BaiduNetdiskDownload/smoking_calling_weight/exp2052/weights/best.pt"


# def get_model():
#     device = select_device('')
#     half = device.type != 'cpu'
#     model = attempt_load(WEIGHTS, map_location=device)
#     stride = int(model.stride.max())
#     names = model.module.names if hasattr(model, 'module') else model.names
#     if half:
#         model.half()
#     return model, device, half, stride, names
# def pred_img_optimized_async(img0, model, device, imgsz, names, conf_thres, iou_thres, half,
#                                    line_thickness, hide_labels, hide_conf, max_det):
#         img = letterbox(img0, new_shape=imgsz, auto=True)[0]
#         img = img.transpose((2, 0, 1))[::-1]
#         img = np.ascontiguousarray(img)
# 
#         img = torch.from_numpy(img).to(device)
#         img = img.half() if half else img.float()
#         img /= 255.0  # 归一化
#         if img.ndimension() == 3:
#             img = img.unsqueeze(0)
# 
#         im0 = img0.copy()
#         annotator = Annotator(im0, line_width=line_thickness, example=str(names))
# 
#         pred = model(img, augment=False, visualize=False)[0]
#         pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False, max_det=max_det)
# 
#         detections = []
#         det = pred[0]
#         if len(det):
#             det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
#             for *xyxy, conf, cls in reversed(det):
#                 c = int(cls)
#                 detections.append({'coords': tuple(map(int, xyxy)), 'confidence': conf.item()})
# 
#                 label = None if hide_labels else f'{names[c]} {conf:.2f}' if not hide_conf else names[c]
#                 annotator.box_label(xyxy, label, color=colors(c))
# 
#         return im0, detections
cap = cv2.VideoCapture(1)

# 用于计算FPS的变量
frame_count = 0
start_time = time.time()
# cap = cv2.VideoCapture(0)  # 假设使用的是第一个摄像头

def fx():
    global frame_count, start_time
    frame_count = 0
    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time if elapsed_time > 0 else 0
        fps_text = f'FPS: {fps:.2f}'
        cv2.putText(frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        cv2.imshow("A video", frame)

        c = cv2.waitKey(1)
        if c == 27:
            break

        if elapsed_time >= 20: 
            print("终止")  # 运行大约20秒后中断
            break

if __name__ == "__main__":
    profiler = cProfile.Profile()
    profiler.enable()
    
    fx()

    profiler.disable()
    print("============")

    # 保存分析结果到文件
    stats = pstats.Stats(profiler).sort_stats('cumtime')
    stats.dump_stats('profiling_results.prof')
    
    # 导出为csv
    s = io.StringIO()
    stats = pstats.Stats('profiling_results.prof', stream=s)
    stats.sort_stats('cumtime')
    stats.print_stats()
    
    # 获取性能数据作为CSV格式
    profiling_data = s.getvalue()
    # 将数据转换为CSV格式
    profiling_data = "ncalls" + profiling_data.split("ncalls")[-1]
    profiling_data = '\n'.join([','.join(line.rstrip().split(None, 5)) for line in profiling_data.split('\n')])
    
    # 将CSV数据保存到文件
    with open('profiling_results.csv', 'w') as f:
        f.write(profiling_data)

    cap.release()
    cv2.destroyAllWindows()

终止


In [6]:
import pandas as pd
 
# 读取CSV
df = pd.read_csv('profiling_results.csv')
sorted_df = df[['cumtime', 'filename:lineno(function)']].sort_values(by='cumtime', ascending=False)

sorted_df

,cumtime,filename:lineno(function)
0,20.017,C:\Users\wsh\AppData\Local\Temp\ipykernel_2156...
1,12.773,{method 'read' of 'cv2.VideoCapture' objects}
2,6.920,{waitKey}
3,0.237,D:\Smoke\SmokeDetectionMasterPython\utils\gene...
4,0.226,{imshow}
5,0.051,{putText}
6,0.007,{method 'encode' of 'str' objects}
7,0.001,{built-in method time.time}
21,0.000,D:\conda_home\envs\yolov5-master\lib\threading...
19,0.000,{method 'acquire' of '_thread.lock' objects}


In [6]:
import os
import cv2
import gc
from multiprocessing import Process, Manager

# 向共享缓冲栈中写入数据:
def write(stack, cam, top: int) -> None:
    """
    :param cam: 摄像头参数
    :param stack: Manager.list对象
    :param top: 缓冲栈容量
    :return: None
    """
    print('Process to write: %s' % os.getpid())
    cap = cv2.VideoCapture(cam)
    while True:
        _, img = cap.read()
        if _:
            stack.append(img)
            # 每到一定容量清空一次缓冲栈
            # 利用gc库，手动清理内存垃圾，防止内存溢出
            if len(stack) >= top:
                del stack[:]
                gc.collect()

# 在缓冲栈中读取数据:
def read(stack) -> None:
    print('Process to read: %s' % os.getpid())
    while True:
        if len(stack) != 0:
            value = stack.pop()
            cv2.imshow("img", value)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

if __name__ == '__main__':
    # 父进程创建缓冲栈，并传给各个子进程：
    q = Manager().list()
    pw = Process(target=write, args=(q, 0, 100))
    pr = Process(target=read, args=(q,))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()

    # 等待pr结束:
    pr.join()

    # pw进程里是死循环，无法等待其结束，只能强行终止:
    pw.terminate()

In [18]:
import av
import cv2

def open_camera_and_display():
    # 使用 DirectShow 打开摄像头
    container = av.open('video=RS 6088Cam', format='dshow', mode='r')
    stream = container.streams.video[0]
    stream.thread_type = 'AUTO'  # 使用多线程解码

    try:
        for frame in container.decode(stream):
            img = frame.to_image()
            img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)  # 将PIL图像转换为OpenCV图像

            cv2.imshow('Camera', img_cv)  # 在窗口中显示图像

            if cv2.waitKey(1) & 0xFF == ord('q'):  # 按q退出
                break
    finally:
        container.close()  # 关闭容器释放资源
        cv2.destroyAllWindows()  # 销毁所有OpenCV窗口

open_camera_and_display()


deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did 

In [33]:
async def saveImageWithPath(territoryId:int)-> LiteralString | str | bytes:
    territory_dir = os.path.join(base_dir, str(territoryId))
    os.makedirs(territory_dir, exist_ok=True)  # 如果不存在则创建
    
    # 当前年份
    year = str(time.localtime().tm_year)
    year_dir = os.path.join(territory_dir, year)
    os.makedirs(year_dir, exist_ok=True)  # 如果不存在则创建
    
    # 当前日期
    month = str(time.localtime().tm_mon)
    day = str(time.localtime().tm_mday)
    date_dir = os.path.join(year_dir, f"{month}-{day}")
    os.makedirs(date_dir, exist_ok=True)  # 如果不存在则创建
    
    # 图片文件名
    file_name = f"smoke_face_{territoryId}_{int(time.time())}.jpg"
    return os.path.join(date_dir, file_name)


KeyboardInterrupt: 

In [2]:
import cv2
cv2.cuda.printCudaDeviceInfo(0)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\include\opencv2/core/private.cuda.hpp:106: error: (-216:No CUDA support) The library is compiled without CUDA support in function 'throw_no_cuda'


In [3]:
import asyncio
import cv2
import time
from concurrent.futures import ThreadPoolExecutor

# 初始化摄像头和线程池
cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)
executor = ThreadPoolExecutor(max_workers=2)

async def capture_frame(camera, camera_id):
    loop = asyncio.get_event_loop()
    frame_count = 0
    start_time = time.time()
    while True:
        # 读取帧
        frame = await loop.run_in_executor(executor, camera.read)
        ret, frame = frame
        if not ret:
            print(f"Camera {camera_id}: Failed to capture frame.")
            continue

        # 编码为JPEG
        ret, jpeg = cv2.imencode('.jpg', frame)
        if not ret:
            print(f"Camera {camera_id}: Failed to encode frame to JPEG.")
            continue

        frame_count += 1
        # 每秒计算帧率
        if time.time() - start_time >= 1.0:
            fps = frame_count / (time.time() - start_time)
            if fps<25 or 80<fps<90:
                 print(f"Camera {camera_id}: {fps} FPS")
            frame_count = 0
            start_time = time.time()

async def main():
    # 启动摄像头帧读取任务
    task0 = asyncio.create_task(capture_frame(cap0, 0))
    task1 = asyncio.create_task(capture_frame(cap1, 1))

    # 等待所有任务完成
    await asyncio.gather(task0, task1)

# 如果已在一个事件循环中，使用以下方式启动任务
await main()  # 在Jupyter Notebook中，直接使用这一行来启动main函数
#

Camera 1: 84.96175580145045 FPS
Camera 1: 85.24869111370607 FPS
Camera 1: 89.6150719793368 FPS
Camera 1: 82.79295366026342 FPS
Camera 1: 81.84420677497592 FPS
Camera 1: 84.15009349332706 FPS
Camera 1: 87.07440841220424 FPS
Camera 1: 83.97903679572937 FPS
Camera 1: 84.60819602219128 FPS
Camera 1: 87.42210548006528 FPS
Camera 1: 85.2620911813065 FPS
Camera 1: 88.2859543728376 FPS
Camera 1: 88.52899823294445 FPS
Camera 1: 86.82609129120412 FPS
Camera 1: 87.6665817945512 FPS
Camera 1: 85.89711115683271 FPS
Camera 1: 87.35360104663742 FPS
Camera 1: 83.03340599981617 FPS
Camera 1: 89.93779431788535 FPS
Camera 1: 85.92987204741995 FPS
Camera 1: 86.67535262353047 FPS
Camera 1: 87.61310151428044 FPS
Camera 1: 86.52312586293851 FPS
Camera 1: 86.54011615538896 FPS
Camera 1: 85.22301103071817 FPS


CancelledError: 

In [4]:


import onnxruntime as ort
import numpy as np 
from time import time
print(ort.get_available_providers())  # ['CPUExecutionProvider']  cpu版本
start = time()
# 初始化session
so = ort.SessionOptions()
onnx_session=ort.InferenceSession('D:\\Smoke\\yolov5\\yolov5\\best.onnx',so)
# 准备输入数据
input_data = np.random.rand(1, 3, 224, 224).astype(np.float32)
# 推理
ort_inputs = {onnx_session.get_inputs()[0].name: input_data}
ort_outputs = onnx_session.run(None, ort_inputs)

# 输出结果
speed = time()-start
print('this case use {:.3f} s'.format(speed))
print("len of out = ",len(ort_outputs))
print(ort_outputs[0].shape)


['AzureExecutionProvider', 'CPUExecutionProvider']


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Unexpected input data type. Actual: (tensor(float)) , expected: (tensor(float16))

In [1]:
import torch

print(torch.__version__)

print(torch.version.cuda)

print(torch.backends.cudnn.version())

2.2.0+cu118
11.8
8700


In [8]:
import asyncio
async def do(time:int)->None:
    await asyncio.sleep(10)
    print("执行完成")

asyncio.run(do)

RuntimeError: asyncio.run() cannot be called from a running event loop